In [ ]:
from datetime import datetime
import logging
import random
import time

from bs4 import BeautifulSoup
import psycopg2
from selenium import webdriver

def log():
    formatter = logging.Formatter('%(asctime)s - [%(levelname)s] - %(message)s')
    hdlr = logging.StreamHandler()
    hdlr.setFormatter(formatter)
    logger = logging.getLogger() 
    logger.setLevel(logging.INFO)
    logger.addHandler(hdlr)
    return logger

class Initial:
    def __init__(self):
        pass
        
    def url(self):
        url_pool = {'info':'https://goodinfo.tw/StockInfo/StockBzPerformance.asp?STOCK_ID='
                   ,'YoY':'https://goodinfo.tw/StockInfo/StockDividendPolicy.asp?STOCK_ID='
                   ,'price':'https://www.twse.com.tw/exchangeReport/STOCK_DAY_AVG?response=html&date=&stockNo='
                   }
        return url_pool
    
    def trigger_driver(self):
        driver = webdriver.Chrome('G:\Anaconda\Tools\chromedriver.exe')
        return driver
        
class Flag:
    
    def __init__(self):
        self.id = 821
        
    def search(self):
        i = Initial()
        dvr = i.trigger_driver()
        url_pool = i.url()
        logger.info('Start to search stock id.')
        for key, value in url_pool.items():
            if key == 'info':
                list_sql = "select stock_id from stock_info where flag in ('1','2');"
                db = Database()
                exist = db.get(list_sql)
                while self.id < 10000:
                    self.id += 1
                    s_id = str(self.id).zfill(4)
                    if s_id not in exist[0]:
                        url = value + s_id
                        logger.info('parse {}'.format(url))
                        dvr.get(url)
                        soup = BeautifulSoup(dvr.page_source,'lxml')
                        hk_sql = "delete from stock_info where stock_id = '{}';".format(s_id)
                        try:
                            name = soup.select('.std_tbl tr > td')[1].text.split('\xa0')[1]
                            dml_sql = "insert into stock_info (stock_id, company, flag) values('{}','{}',3);".format(s_id,name)
                        except:
                            dml_sql = "insert into stock_info (stock_id, company, flag) values('{}',null,0);".format(s_id)
                
                        try:
                            d = Database()
                            d.run(hk_sql)
                            d.run(dml_sql)
                            d.logout()
                        except:
                            logger.error('Database error')
                        
                        if '瀏覽量異常' in str(soup):
                            f.note(self.id)
                            break
                        
                        t = random.randint(30,60)
                        time.sleep(t+120)
            
        dvr.quit()
    
    def note(self, count):
        try:
            n = Database()
            if count == 0:
                n.run("delete from check_blocking;")
                note_sql = "insert into check_blocking select 'start', current_timestamp;"
                n.run(note_sql)
            else:
                note_sql = "insert into check_blocking select '{}', current_timestamp;".format(count)
                n.run(note_sql)

            n.logout()
        except:
            None
            
class Crawler:
    
    def __init__(self):
        f.note(0)
        try:
            d = Database()
            self.id_pool = d.get("select stock_id, company from stock_info where flag='1' and stock_id > '8007' order by stock_id;")
            d.logout()
        except:
            None
        
    def yoy(self):
        i = Initial()
        dvr = i.trigger_driver()
        url_pool = i.url()
        
        for key, value in url_pool.items():
            if key == 'YoY':
                logger.info('Start to parse YoY information of stocks.')
                d = Database()
                for number in self.id_pool:
                    stock_id = number[0]
                    url = value + stock_id
                    logger.info('parse {}'.format(url))
                    dvr.get(url)
                    soup = BeautifulSoup(dvr.page_source,'lxml')
                    
                    #if '瀏覽量異常' in str(soup):
                    #    f.note(stock_id)
                    #    break
                    
                    extr = Extract()
                    
                    per = extr.data(soup, '.std_tbl > tbody > tr:nth-of-type(5) > td:nth-of-type(6)', 'float')
                    pbr = extr.data(soup, '.std_tbl > tbody > tr:nth-of-type(5) > td:nth-of-type(7)', 'float')
                    d.run("delete from stock_per_pbr where stock_id = '{}';".format(stock_id))
                    d.run("insert into stock_per_pbr values('{}', {}, {});".format(stock_id, per, pbr))
                    
                    for count in range(1,5):
                        cash_dividend = extr.data(soup, '.solid_1_padding_4_0_tbl > tbody > tr:nth-of-type({}) \
                                                         > td:nth-of-type(4)'.format(count), 'float')
                        stock_dividend = extr.data(soup, '.solid_1_padding_4_0_tbl > tbody > tr:nth-of-type({}) \
                                                         > td:nth-of-type(7)'.format(count), 'float')
                        eps = extr.data(soup, '.solid_1_padding_4_0_tbl > tbody > tr:nth-of-type({}) \
                                               > td:nth-of-type(23)'.format(count), 'float')
                        year = extr.data(soup, '.solid_1_padding_4_0_tbl > tbody > tr:nth-of-type({}) \
                                                > td:nth-of-type(22)'.format(count))
                        spls_aption = extr.data(soup, '.solid_1_padding_4_0_tbl > tbody > tr:nth-of-type({}) \
                                                       > td:nth-of-type(26)'.format(count), 'float')
                        d.run("delete from stock_yoy_revenue where stock_id = '{}' and year = '{}';".format(stock_id, year))
                        d.run("insert into stock_yoy_revenue values('{}','{}',\
                              {},{},{},{})".format(stock_id, year, eps, stock_dividend, cash_dividend, spls_aption))
                                    
                    t = random.randint(30,60)
                    time.sleep(t+180)
                
                d.logout()
                logger.info('The parsing of stocks\' YoY information finished.')

        dvr.quit()
    
    def price(self):
        pri_i = Initial()
        pri_dvr = pri_i.trigger_driver()
        pri_url_pool = pri_i.url()
        
        for pri_key, pri_value in pri_url_pool.items():
            if pri_key == 'price':
                logger.info('Start to parse price of stocks.')
                pri_db = Database()
                for pri_number in self.id_pool:
                    stock_no = pri_number[0]
                    pri_date = 'date=' + datetime.strftime(datetime.today(),'%Y%m%d')
                    pri_url = pri_value.replace('date=',pri_date) + stock_no
                    logger.info('parse {}'.format(pri_url))
                    pri_dvr.get(pri_url)
                    pri_soup = BeautifulSoup(pri_dvr.page_source,'lxml')
                    trading_day = pri_soup.select('table > tbody > tr > td:nth-of-type(1)')
                    pri_extr = Extract()
                    for c in range(0, len(trading_day)-1):
                        ad = pri_extr.price_data(pri_soup, 'table > tbody > tr > td:nth-of-type(1)', 'convert_to_AD', c)
                        p = pri_extr.price_data(pri_soup, 'table > tbody > tr > td:nth-of-type(2)', 'float', c)
                        pri_db.run("delete from stock_price where stock_id='{}' and trading_day='{}';".format(stock_no, ad))
                        pri_db.run("insert into stock_price values('{}','{}',{});".format(stock_no, ad, p))
                    
                    t = random.randint(30,60)
                    time.sleep(t+150)
                
                pri_db.logout()
                logger.info('The parsing of stocks\' price finished.')
        
        pri_dvr.quit()

class Extract:
    
    def __init__(self):
        pass
    
    def data(self, bs4_soup, selector, transfer=None):
        if transfer == 'float':
            try:
                return float(bs4_soup.select(selector)[0].text)
            except:
                return 'null'
    
    def price_data(self, bs4_soup, selector, transfer=None, count=None):
        if transfer == 'convert_to_AD':
            try:
                ad = bs4_soup.select(selector)[count].text
                pat = ad.split('/')[0]
                new_pat = str(int(pat) + 1911)
                return ad.replace(pat, new_pat)
            except:
                return '9999'
        elif transfer == 'float':
            try:
                return float(bs4_soup.select(selector)[count].text)
            except:
                return -1
                
class Database:
    
    def __init__(self):
        self.con = psycopg2.connect(database='stock', user='postgres', password='postgres', host='localhost', port='5432')
        self.cur = self.con.cursor()
    
    def run(self, sql):
        self.cur.execute(sql)
        self.con.commit()
        
    def get(self, sql):
        self.cur.execute(sql)
        data = self.cur.fetchall()
        self.con.commit()
        return data   # return tuple
    
    def logout(self):
        try:
            self.con.close()
        except:
            logger.info('already logged out!')

if __name__ == '__main__':
    logger = log()
    f = Flag()
    f.note(0)
    f.search()
    #c = Crawler()
    #c.price()
    
    #1224